In [1]:
!pip install transformers datasets evaluate accelerate trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
login()

In [3]:
from datasets import load_dataset
rewild = load_dataset("chardizard/modified-rewild")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/323 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/236M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/212M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/319842 [00:00<?, ? examples/s]

In [4]:
rewild = rewild.rename_column("response", "completion")['train']

In [5]:
rewild

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 319842
})

In [ ]:
# model = GPTNeoXForCausalLM.from_pretrained(
#   "EleutherAI/pythia-70m-deduped",
#   revision="step3000",
#   cache_dir="./pythia-70m-deduped/step3000",
# )

# dytmodel = GPTNeoXForCausalLM.from_pretrained(
#   "EleutherAI/pythia-70m-deduped",
#   revision="step3000",
#   cache_dir="./pythia-70m-deduped/step3000",
# )

# tokenizer = AutoTokenizer.from_pretrained(
#   "EleutherAI/pythia-70m-deduped",
#   revision="step3000",
#   cache_dir="./pythia-70m-deduped/step3000",
# )

# inputs = tokenizer("Hello, I am", return_tensors="pt")
# tokens = model.generate(**inputs)
# tokenizer.decode(tokens[0])


In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-14m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-14m")
dytmodel = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-14m")

tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/53.3M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length

1000000000000000019884624838656

In [7]:
import torch
import torch.nn as nn
from timm.layers import LayerNorm2d


class DynamicTanh(nn.Module):
    def __init__(self, normalized_shape, channels_last, alpha_init_value=0.5):
        super().__init__()
        self.normalized_shape = normalized_shape
        self.alpha_init_value = alpha_init_value
        self.channels_last = channels_last

        self.alpha = nn.Parameter(torch.ones(1) * alpha_init_value)
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))

    def forward(self, x):
        x = torch.tanh(self.alpha * x)
        if self.channels_last:
            x = x * self.weight + self.bias
        else:
            x = x * self.weight[:, None, None] + self.bias[:, None, None]
        return x

    def extra_repr(self):
        return f"normalized_shape={self.normalized_shape}, alpha_init_value={self.alpha_init_value}, channels_last={self.channels_last}"


def convert_ln_to_dyt(module):
    module_output = module
    if isinstance(module, nn.LayerNorm):
        module_output = DynamicTanh(module.normalized_shape, not isinstance(module, LayerNorm2d))
    for name, child in module.named_children():
        module_output.add_module(name, convert_ln_to_dyt(child))
    del module
    return module_output

In [8]:
dytmodel = convert_ln_to_dyt(dytmodel)

In [ ]:
dytmodel, model

(GPTNeoXForCausalLM(
   (gpt_neox): GPTNeoXModel(
     (embed_in): Embedding(50304, 128)
     (emb_dropout): Dropout(p=0.0, inplace=False)
     (layers): ModuleList(
       (0-5): 6 x GPTNeoXLayer(
         (input_layernorm): DynamicTanh(normalized_shape=(128,), alpha_init_value=0.5, channels_last=True)
         (post_attention_layernorm): DynamicTanh(normalized_shape=(128,), alpha_init_value=0.5, channels_last=True)
         (post_attention_dropout): Dropout(p=0.0, inplace=False)
         (post_mlp_dropout): Dropout(p=0.0, inplace=False)
         (attention): GPTNeoXAttention(
           (query_key_value): Linear(in_features=128, out_features=384, bias=True)
           (dense): Linear(in_features=128, out_features=128, bias=True)
         )
         (mlp): GPTNeoXMLP(
           (dense_h_to_4h): Linear(in_features=128, out_features=512, bias=True)
           (dense_4h_to_h): Linear(in_features=512, out_features=128, bias=True)
           (act): GELUActivation()
         )
       )
   

In [9]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rhz2020 (rhz2020-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
wandb.init(project="Rewild", name = "17m")

In [11]:
from trl import SFTConfig, SFTTrainer

In [12]:
login()

In [15]:
training_args = SFTConfig(
    max_length=512,
    output_dir="/tmp",
    num_train_epochs=3,
    use_cpu= False,
    dataloader_num_workers= 4,
    push_to_hub=True,
    save_strategy='epoch'
)

trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=rewild,
        processing_class=tokenizer,
    )
trainer.train()

Converting train dataset to ChatML:   0%|          | 0/319842 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/319842 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/319842 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:597: UserWarning: Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.
  warnings.warn(


Truncating train dataset:   0%|          | 0/319842 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
500,3.383100
1000,3.247600
1500,3.196500
2000,3.156300
2500,3.133100
3000,3.123000
3500,3.080900
4000,3.040700
4500,3.020400
5000,3.049500


KeyboardInterrupt: 

In [ ]:
trainer.push_to_hub(repo_id="pythia17m")

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


CommitInfo(commit_url='https://huggingface.co/chardizard/tmp/commit/67a321482e7da06c6298cc688ee724f6078f9341', commit_message='End of training', commit_description='', oid='67a321482e7da06c6298cc688ee724f6078f9341', pr_url=None, repo_url=RepoUrl('https://huggingface.co/chardizard/tmp', endpoint='https://huggingface.co', repo_type='model', repo_id='chardizard/tmp'), pr_revision=None, pr_num=None)

In [ ]:
wandb.init(project="Rewild", name = "dyt17m")

train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇████
train/grad_norm,▆▄▅▄▆█▄▄▃▅▃▄▃▃▃▃▅▄▃▄▃▅▄▆▂▄▃▂▄▃▄▂▄▃▃▃▁▂▃▂
train/learning_rate,██▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,█▆▆▆▆▅▆▅▅▆▄▅▅▅▄▃▃▃▂▃▃▃▃▃▃▂▃▃▃▂▂▂▁▃▁▂▁▂▂▁
train/mean_token_accuracy,▁▁▂▂▂▁▃▂▂▂▃▄▅▃▄▅▅▅▅▆▆▅▅▇▇▇▇▇▇▅▇▇▆█▇▇███▇
train/num_tokens,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███
total_flos,2.24794593017856e+16
train/epoch,3
train/global_step,119943
train/grad_norm,12.37817


In [ ]:
dyttraining_args = SFTConfig(
    max_length=512,
    output_dir="/tmp",
    num_train_epochs=3,
    use_cpu= False,
    dataloader_num_workers= 4,
    push_to_hub=True,
    save_strategy='epoch'
)
dyttrainer = SFTTrainer(
        model=dytmodel,
        args=dyttraining_args,
        train_dataset=rewild,
        processing_class=tokenizer,
    )
dyttrainer.train()

Step,Training Loss
500,6.773800
1000,6.745400
1500,6.723600
2000,6.701100
2500,6.678600
3000,6.649700
3500,6.659900
4000,6.648100
4500,6.621600
5000,6.609800


TrainOutput(global_step=119943, training_loss=6.04257693180628, metrics={'train_runtime': 4764.8892, 'train_samples_per_second': 201.374, 'train_steps_per_second': 25.172, 'total_flos': 2.2479497608330656e+16, 'train_loss': 6.04257693180628})

In [ ]:
dyttrainer.push_to_hub(repo_id="pythia17mDyt")